# Phase 2: Enhanced Feature Engineering

**Objective:** Improve model performance by adding:
1. **Glassnode on-chain metrics** - Regime/sentiment indicators
2. **Technical indicators** - RSI, MACD, Bollinger Bands

## Key Hypothesis
The challenge mentions "latent regime dynamics". On-chain metrics like MVRV, Fear & Greed, 
and SOPR may help identify these regimes and improve our predictions.

---

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ML imports
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Add src to path
import sys
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from data import load_all_data
from features import compute_rolling_returns, compute_rolling_volatility, compute_rsi, compute_bollinger_bands
from labels import create_cost_adjusted_labels, analyze_label_distribution
from metrics import compute_all_metrics
from backtesting import compute_strategy_returns, compute_portfolio_returns, compute_equity_curve

## 1. Load Data

In [ ]:
# Load all data
trade_log, prices, glassnode = load_all_data()

# Align trade_log and prices
common_idx = trade_log.index.intersection(prices.index)
common_assets = trade_log.columns.intersection(prices.columns)

signals = trade_log.loc[common_idx, common_assets]
prices_aligned = prices.loc[common_idx, common_assets]

print(f"Signals: {signals.shape}")
print(f"Prices: {prices_aligned.shape}")
print(f"Glassnode: {glassnode.shape}")
print(f"\nPeriod: {signals.index.min()} to {signals.index.max()}")

---

## 2. Glassnode Feature Selection

### Key On-Chain Metrics

| Feature | What it Measures | Interpretation |
|---------|------------------|----------------|
| MVRV Z-Score | Market vs Realized Value | >3 = overvalued (top), <0 = undervalued (bottom) |
| Fear & Greed | Market Sentiment | 0-25 = fear (buy signal), 75-100 = greed (sell signal) |
| SOPR | Profit/Loss Ratio | >1 = profit taking, <1 = capitulation |
| Puell Multiple | Miner Revenue | >4 = miners selling (bearish), <0.5 = accumulation |
| % Supply in Profit | How much BTC is profitable | Low = capitulation, High = euphoria |

In [ ]:
# Select key Glassnode features
GLASSNODE_FEATURES = [
    # Valuation
    'btc_mvrv_z_score',
    'btc_puell_multiple',
    'reserve_risk',
    
    # Sentiment
    'btc_fear_greed_index',
    'btc_adjusted_sopr',
    
    # On-chain activity
    'btc_percent_upply_in_profit',
    'btc_network_value_to_transactions_signal',
    
    # Derivatives
    'btc_futures_perpetual_funding_rate_mean',
    
    # Supply dynamics
    'vocdd',
    'mvocdd',
]

# Check availability
available_gn = [f for f in GLASSNODE_FEATURES if f in glassnode.columns]
print(f"Available Glassnode features: {len(available_gn)}/{len(GLASSNODE_FEATURES)}")
for f in available_gn:
    print(f"  - {f}")

In [ ]:
# Prepare Glassnode features
# Note: Glassnode is daily, we need to align with our 3-hour data

gn_selected = glassnode[available_gn].copy()

# Make timezone-naive for alignment
signals_naive = signals.copy()
signals_naive.index = signals_naive.index.tz_localize(None)

# Resample Glassnode to match signals (forward fill daily to 3-hourly)
# First, get the date part of each signal timestamp
gn_aligned = pd.DataFrame(index=signals_naive.index)

for col in available_gn:
    # Create a series indexed by date
    gn_series = gn_selected[col]
    
    # For each timestamp in signals, get the Glassnode value for that date
    aligned_values = []
    for ts in signals_naive.index:
        date = ts.normalize()  # Get just the date part
        if date in gn_series.index:
            aligned_values.append(gn_series.loc[date])
        else:
            # Find the most recent available date
            available_dates = gn_series.index[gn_series.index <= date]
            if len(available_dates) > 0:
                aligned_values.append(gn_series.loc[available_dates[-1]])
            else:
                aligned_values.append(np.nan)
    
    gn_aligned[col] = aligned_values

print(f"Aligned Glassnode features: {gn_aligned.shape}")
print(f"Missing values: {gn_aligned.isna().sum().sum()}")

In [ ]:
# Visualize key Glassnode features
fig, axes = plt.subplots(3, 2, figsize=(14, 10))

features_to_plot = [
    ('btc_mvrv_z_score', 'MVRV Z-Score'),
    ('btc_fear_greed_index', 'Fear & Greed Index'),
    ('btc_adjusted_sopr', 'Adjusted SOPR'),
    ('btc_puell_multiple', 'Puell Multiple'),
    ('btc_percent_upply_in_profit', '% Supply in Profit'),
    ('btc_futures_perpetual_funding_rate_mean', 'Funding Rate Mean'),
]

for ax, (col, title) in zip(axes.flat, features_to_plot):
    if col in gn_aligned.columns:
        ax.plot(gn_aligned.index, gn_aligned[col], alpha=0.7)
        ax.set_title(title)
        ax.tick_params(axis='x', rotation=45)
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 3. Technical Indicators

In [ ]:
# Compute technical indicators for each asset
def compute_macd(prices, fast=96, slow=208, signal=72):
    """
    Compute MACD (Moving Average Convergence Divergence).
    Default periods adjusted for 3-hour data:
    - Fast: 12 days = 96 periods
    - Slow: 26 days = 208 periods  
    - Signal: 9 days = 72 periods
    """
    features = pd.DataFrame(index=prices.index)
    
    for col in prices.columns:
        ema_fast = prices[col].ewm(span=fast, adjust=False).mean()
        ema_slow = prices[col].ewm(span=slow, adjust=False).mean()
        
        macd_line = ema_fast - ema_slow
        signal_line = macd_line.ewm(span=signal, adjust=False).mean()
        histogram = macd_line - signal_line
        
        # Normalize by price to make comparable across assets
        features[f'{col}_macd'] = macd_line / prices[col]
        features[f'{col}_macd_signal'] = signal_line / prices[col]
        features[f'{col}_macd_hist'] = histogram / prices[col]
    
    return features

# Compute all technical indicators
print("Computing technical indicators...")

# Price-based features (from Phase 1)
return_features = compute_rolling_returns(prices_aligned, windows=[1, 8, 56, 224])
vol_features = compute_rolling_volatility(prices_aligned, windows=[56, 224])

# New technical indicators
rsi_features = compute_rsi(prices_aligned, window=112)  # 14 days
bb_features = compute_bollinger_bands(prices_aligned, window=160)  # 20 days
macd_features = compute_macd(prices_aligned)

print(f"Return features: {return_features.shape}")
print(f"Volatility features: {vol_features.shape}")
print(f"RSI features: {rsi_features.shape}")
print(f"Bollinger Band features: {bb_features.shape}")
print(f"MACD features: {macd_features.shape}")

---

## 4. Combine All Features

In [ ]:
# Combine all price-based features
price_features = pd.concat([
    return_features,
    vol_features,
    rsi_features,
    bb_features,
    macd_features
], axis=1)

print(f"Combined price features: {price_features.shape}")
print(f"\nFeature types:")
print(f"  - Returns: {len([c for c in price_features.columns if 'return' in c])}")
print(f"  - Volatility: {len([c for c in price_features.columns if 'volatility' in c])}")
print(f"  - RSI: {len([c for c in price_features.columns if 'rsi' in c])}")
print(f"  - Bollinger: {len([c for c in price_features.columns if 'bb' in c])}")
print(f"  - MACD: {len([c for c in price_features.columns if 'macd' in c])}")

---

## 5. Create Labels

In [ ]:
# Create labels (same as Phase 1)
HORIZON = 8  # 1 day

labels = create_cost_adjusted_labels(
    prices_aligned,
    signals,
    horizon=HORIZON,
    entry_cost=0.001,
    exit_cost=0.001
)

label_stats = analyze_label_distribution(labels, signals)
print(f"Labels: {labels.shape}")
print(f"Positive rate: {label_stats['overall']['positive_rate']*100:.1f}%")

---

## 6. Prepare Stacked Data with All Features

In [ ]:
def prepare_enhanced_data(price_features, glassnode_features, labels, signals):
    """
    Prepare stacked data with both price-based and Glassnode features.
    
    Price features are asset-specific (renamed to generic).
    Glassnode features are global (same for all assets).
    """
    data_rows = []
    
    # Make signals index timezone-naive for comparison
    signals_naive = signals.copy()
    signals_naive.index = signals_naive.index.tz_localize(None)
    
    labels_naive = labels.copy()
    labels_naive.index = labels_naive.index.tz_localize(None)
    
    price_features_naive = price_features.copy()
    price_features_naive.index = price_features_naive.index.tz_localize(None)
    
    for timestamp in labels_naive.index:
        if timestamp not in price_features_naive.index:
            continue
        if timestamp not in glassnode_features.index:
            continue
            
        for asset in labels_naive.columns:
            # Only include rows where signal = 1
            if signals_naive.loc[timestamp, asset] != 1:
                continue
                
            label_val = labels_naive.loc[timestamp, asset]
            if pd.isna(label_val):
                continue
            
            # Get asset-specific price features
            asset_cols = [c for c in price_features_naive.columns if c.startswith(asset + '_')]
            if not asset_cols:
                continue
                
            price_row = price_features_naive.loc[timestamp, asset_cols]
            if price_row.isna().any():
                continue
            
            # Get Glassnode features (global)
            gn_row = glassnode_features.loc[timestamp]
            if gn_row.isna().any():
                continue
            
            # Rename price features to be asset-agnostic
            renamed_price = {}
            for col in asset_cols:
                new_name = col.replace(asset + '_', '')
                renamed_price[new_name] = price_row[col]
            
            # Combine all features
            row_data = {
                'timestamp': timestamp,
                'asset': asset,
                'label': label_val,
                **renamed_price,
                **gn_row.to_dict()
            }
            data_rows.append(row_data)
    
    df = pd.DataFrame(data_rows)
    df = df.set_index(['timestamp', 'asset'])
    
    y = df['label']
    X = df.drop('label', axis=1)
    
    return X, y

X, y = prepare_enhanced_data(price_features, gn_aligned, labels, signals)
print(f"Enhanced data: X = {X.shape}, y = {y.shape}")
print(f"\nFeature columns ({len(X.columns)}):")
for i, col in enumerate(X.columns):
    print(f"  {i+1:2d}. {col}")

In [ ]:
# Check for any remaining NaN
print(f"NaN in X: {X.isna().sum().sum()}")
print(f"NaN in y: {y.isna().sum()}")

if X.isna().sum().sum() > 0:
    print("\nColumns with NaN:")
    nan_cols = X.isna().sum()
    print(nan_cols[nan_cols > 0])

---

## 7. Walk-Forward Validation

In [ ]:
def walk_forward_cv(X, y, train_size, test_size, step_size=None):
    """Generate walk-forward cross-validation splits."""
    if step_size is None:
        step_size = test_size
    
    timestamps = X.index.get_level_values('timestamp').unique().sort_values()
    n_timestamps = len(timestamps)
    
    fold = 0
    start = 0
    
    while start + train_size + test_size <= n_timestamps:
        train_end = start + train_size
        test_end = train_end + test_size
        
        train_timestamps = timestamps[start:train_end]
        test_timestamps = timestamps[train_end:test_end]
        
        train_idx = X.index.get_level_values('timestamp').isin(train_timestamps)
        test_idx = X.index.get_level_values('timestamp').isin(test_timestamps)
        
        fold_info = {
            'fold': fold,
            'train_start': train_timestamps[0],
            'train_end': train_timestamps[-1],
            'test_start': test_timestamps[0],
            'test_end': test_timestamps[-1],
        }
        
        yield train_idx, test_idx, fold_info
        
        start += step_size
        fold += 1

# CV parameters
n_timestamps = X.index.get_level_values('timestamp').nunique()
TRAIN_SIZE = int(n_timestamps * 0.6)
TEST_SIZE = int(n_timestamps * 0.1)
STEP_SIZE = TEST_SIZE

print(f"Total timestamps: {n_timestamps}")
print(f"Train: {TRAIN_SIZE}, Test: {TEST_SIZE}, Step: {STEP_SIZE}")

In [ ]:
# Train with Logistic Regression (same as Phase 1 for fair comparison)
def train_model(X_train, y_train):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)
    
    model = LogisticRegression(
        C=1.0,
        class_weight='balanced',
        max_iter=1000,
        random_state=42
    )
    model.fit(X_scaled, y_train)
    
    return model, scaler

# Run walk-forward validation
all_predictions = []
fold_results = []

for train_idx, test_idx, fold_info in walk_forward_cv(X, y, TRAIN_SIZE, TEST_SIZE, STEP_SIZE):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    model, scaler = train_model(X_train, y_train)
    
    X_test_scaled = scaler.transform(X_test)
    y_prob = model.predict_proba(X_test_scaled)[:, 1]
    y_pred = (y_prob > 0.5).astype(int)
    
    pred_df = pd.DataFrame({
        'probability': y_prob,
        'prediction': y_pred,
        'actual': y_test.values
    }, index=y_test.index)
    all_predictions.append(pred_df)
    
    fold_info['accuracy'] = accuracy_score(y_test, y_pred)
    fold_info['auc'] = roc_auc_score(y_test, y_prob) if len(np.unique(y_test)) > 1 else 0.5
    fold_results.append(fold_info)
    
    print(f"Fold {fold_info['fold']}: Acc={fold_info['accuracy']:.3f}, AUC={fold_info['auc']:.3f}")

predictions_df = pd.concat(all_predictions)

# Summary
results_df = pd.DataFrame(fold_results)
print(f"\nMean Accuracy: {results_df['accuracy'].mean():.3f} (+/- {results_df['accuracy'].std():.3f})")
print(f"Mean AUC: {results_df['auc'].mean():.3f} (+/- {results_df['auc'].std():.3f})")

---

## 8. Evaluate Strategy Performance

In [ ]:
def create_filtered_signals(predictions_df, signals, threshold=0.5):
    """Apply ML filter to baseline signals."""
    # Make signals timezone-naive
    signals_naive = signals.copy()
    signals_naive.index = signals_naive.index.tz_localize(None)
    
    pred_timestamps = predictions_df.index.get_level_values('timestamp').unique()
    filtered = signals_naive.loc[pred_timestamps].copy()
    
    for (timestamp, asset), row in predictions_df.iterrows():
        if asset in filtered.columns and row['probability'] <= threshold:
            filtered.loc[timestamp, asset] = 0
    
    return filtered

# Get test period data
pred_timestamps = predictions_df.index.get_level_values('timestamp').unique()

# Make prices timezone-naive
prices_naive = prices_aligned.copy()
prices_naive.index = prices_naive.index.tz_localize(None)

signals_naive = signals.copy()
signals_naive.index = signals_naive.index.tz_localize(None)

baseline_signals = signals_naive.loc[pred_timestamps]
comparison_prices = prices_naive.loc[pred_timestamps]

# Baseline performance
baseline_returns = compute_strategy_returns(baseline_signals, comparison_prices, transaction_cost=0.001)
baseline_portfolio = compute_portfolio_returns(baseline_returns)
baseline_metrics = compute_all_metrics(baseline_portfolio.dropna())

print("BASELINE (Test Period):")
print(f"  Sharpe Ratio: {baseline_metrics['sharpe_ratio']:.3f}")
print(f"  Total Return: {baseline_metrics['total_return']*100:.2f}%")
print(f"  Max Drawdown: {baseline_metrics['max_drawdown']*100:.2f}%")

In [ ]:
# Test different thresholds
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
threshold_results = []

print("\nML-FILTERED STRATEGY (Enhanced Features):")
print("=" * 60)

for threshold in thresholds:
    filtered_signals = create_filtered_signals(predictions_df, signals, threshold)
    filtered_returns = compute_strategy_returns(filtered_signals, comparison_prices, transaction_cost=0.001)
    filtered_portfolio = compute_portfolio_returns(filtered_returns)
    filtered_metrics = compute_all_metrics(filtered_portfolio.dropna())
    
    n_trades_baseline = baseline_signals.diff().abs().sum().sum() / 2
    n_trades_filtered = filtered_signals.diff().abs().sum().sum() / 2
    
    result = {
        'threshold': threshold,
        'sharpe': filtered_metrics['sharpe_ratio'],
        'total_return': filtered_metrics['total_return'],
        'max_drawdown': filtered_metrics['max_drawdown'],
        'trade_reduction': (n_trades_baseline - n_trades_filtered) / n_trades_baseline * 100
    }
    threshold_results.append(result)
    
    sharpe_change = (result['sharpe'] - baseline_metrics['sharpe_ratio']) / baseline_metrics['sharpe_ratio'] * 100
    print(f"\nThreshold τ = {threshold}:")
    print(f"  Sharpe: {result['sharpe']:.3f} ({sharpe_change:+.1f}% vs baseline)")
    print(f"  Return: {result['total_return']*100:.2f}%")
    print(f"  Max DD: {result['max_drawdown']*100:.2f}%")
    print(f"  Trade Reduction: {result['trade_reduction']:.1f}%")

In [ ]:
# Plot comparison
threshold_df = pd.DataFrame(threshold_results)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sharpe ratio
axes[0].plot(threshold_df['threshold'], threshold_df['sharpe'], 'go-', markersize=10, linewidth=2, label='Phase 2 (Enhanced)')
axes[0].axhline(y=baseline_metrics['sharpe_ratio'], color='r', linestyle='--', linewidth=2, label='Baseline')
axes[0].set_xlabel('Threshold (τ)', fontsize=12)
axes[0].set_ylabel('Sharpe Ratio', fontsize=12)
axes[0].set_title('Sharpe Ratio vs Threshold', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Total return vs trade reduction
axes[1].scatter(threshold_df['trade_reduction'], threshold_df['total_return']*100, 
                c=threshold_df['threshold'], cmap='viridis', s=200, edgecolors='black')
axes[1].axhline(y=baseline_metrics['total_return']*100, color='r', linestyle='--', label='Baseline Return')
axes[1].set_xlabel('Trade Reduction (%)', fontsize=12)
axes[1].set_ylabel('Total Return (%)', fontsize=12)
axes[1].set_title('Return vs Trade Reduction', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Add colorbar
cbar = plt.colorbar(axes[1].collections[0], ax=axes[1])
cbar.set_label('Threshold τ')

plt.tight_layout()
plt.show()

---

## 9. Feature Importance Analysis

In [ ]:
# Train on all data to get feature importance
model_final, scaler_final = train_model(X, y)

importance = pd.DataFrame({
    'feature': X.columns,
    'coefficient': model_final.coef_[0],
    'abs_coefficient': np.abs(model_final.coef_[0])
}).sort_values('abs_coefficient', ascending=False)

print("Top 20 Most Important Features:")
print("=" * 60)
for i, row in importance.head(20).iterrows():
    direction = '+' if row['coefficient'] > 0 else '-'
    # Highlight Glassnode features
    marker = '*' if row['feature'] in GLASSNODE_FEATURES else ' '
    print(f"{marker} {row['feature']:45s} {direction} ({row['abs_coefficient']:.4f})")

print("\n* = Glassnode feature")

In [ ]:
# Plot feature importance
fig, ax = plt.subplots(figsize=(10, 10))

top_features = importance.head(20)
colors = ['green' if c > 0 else 'red' for c in top_features['coefficient']]

# Highlight Glassnode features with different edge
edge_colors = ['blue' if f in GLASSNODE_FEATURES else 'black' for f in top_features['feature']]

bars = ax.barh(range(len(top_features)), top_features['coefficient'], color=colors, alpha=0.7, edgecolor=edge_colors, linewidth=2)
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['feature'])
ax.set_xlabel('Coefficient')
ax.set_title('Top 20 Feature Importance\n(Blue border = Glassnode feature)')
ax.axvline(x=0, color='black', linewidth=0.5)
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

---

## 10. Comparison: Phase 1 vs Phase 2

In [ ]:
# Summary comparison
print("PHASE COMPARISON")
print("=" * 60)
print(f"\n{'Metric':<25} {'Phase 1':>12} {'Phase 2':>12} {'Change':>12}")
print("-" * 60)

# Phase 1 results (from previous notebook - approximate values)
phase1_best_sharpe = 3.93  # at τ=0.6
phase1_best_return = 11.71

# Phase 2 best results
best_idx = threshold_df['sharpe'].idxmax()
phase2_best_sharpe = threshold_df.loc[best_idx, 'sharpe']
phase2_best_return = threshold_df.loc[best_idx, 'total_return'] * 100
phase2_best_threshold = threshold_df.loc[best_idx, 'threshold']

print(f"{'Features':<25} {'4':>12} {len(X.columns):>12} {f'+{len(X.columns)-4}':>12}")
print(f"{'Best Sharpe':<25} {phase1_best_sharpe:>12.3f} {phase2_best_sharpe:>12.3f} {phase2_best_sharpe - phase1_best_sharpe:>+12.3f}")
print(f"{'Best Threshold':<25} {'0.6':>12} {phase2_best_threshold:>12.1f} {'-':>12}")
print(f"{'Return at Best':<25} {phase1_best_return:>11.2f}% {phase2_best_return:>11.2f}% {phase2_best_return - phase1_best_return:>+11.2f}%")

---

## 11. Summary

### What we added:
1. **Glassnode on-chain metrics** (10 features)
   - MVRV Z-Score, Fear & Greed, SOPR, Puell Multiple, etc.
2. **Technical indicators** (15+ features)
   - RSI, Bollinger Bands, MACD

### Key findings:
- [ ] Fill in after running

### Next steps:
- Phase 3: Optimize label horizon and costs
- Phase 4: Try better models (XGBoost)

In [ ]:
# Save results
output_dir = Path.cwd().parent / 'data' / 'processed'
output_dir.mkdir(parents=True, exist_ok=True)

predictions_df.to_csv(output_dir / 'phase2_predictions.csv')
print(f"Saved predictions to {output_dir / 'phase2_predictions.csv'}")